<a href="https://colab.research.google.com/github/avkav/pyspark/blob/main/colab/template_pyspark_basic_commands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

Always execute this cell at the beginning

In [94]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
52 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

# Create a DataFrame

In [95]:
data = [
    (1001, "Ana Perez", 20, "Engineering", 8.5),
    (1002, "Luis Gomez", 22, "Engineering", 9.0),
    (1003, "Maria Lopez", 21, "Engineering", 7.8),
    (1004, "Javier Sanchez", 23, "Architecture", 6.5),
    (1005, "Carmen Rodriguez", 20, "Architecture", 8.2)
]

schema = T.StructType([
    T.StructField("id", T.IntegerType(), True),#El True nos indica que el campo puede tomar valores nulos
    T.StructField("name", T.StringType(), True),
    T.StructField("age", T.IntegerType(), True),
    T.StructField("degree", T.StringType(), True),
    T.StructField("grade", T.FloatType(), True)
])

PySpark tiene una forma de operar que se llama **Lazy execution**, que quiere decir qe solo va a ejecutar las operaciones si se lo han dicho explícitamente. En este caso llamamos a la variable df pero no muestra los datos en formato de tabla. Para ello hay que utilizar el método show.

In [96]:
df = spark.createDataFrame(data,schema)
df
df.show()

+----+----------------+---+------------+-----+
|  id|            name|age|      degree|grade|
+----+----------------+---+------------+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|
|1002|      Luis Gomez| 22| Engineering|  9.0|
|1003|     Maria Lopez| 21| Engineering|  7.8|
|1004|  Javier Sanchez| 23|Architecture|  6.5|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|
+----+----------------+---+------------+-----+



# Select columns

**select** operación de transformación, así que es perezosa (**lazy**) y no se ejecuta hasta que no es estrictamente necesario.

In [97]:
df.select('name','id')

DataFrame[name: string, id: int]

Para visualizar la información, debemos indicar una **acción** como **show** para que se ejecute la operación de seleccionar columnas.

In [98]:
df.select('name','id').show()

+----------------+----+
|            name|  id|
+----------------+----+
|       Ana Perez|1001|
|      Luis Gomez|1002|
|     Maria Lopez|1003|
|  Javier Sanchez|1004|
|Carmen Rodriguez|1005|
+----------------+----+



**F.col** es otra forma de hacer referencia a las columnas de una tabla. No es la forma más concisa de hacer un **select**, pero para otras operaciones sí es necesario utilizarlo.

In [99]:
df.select(F.col('name'),F.col('id')).show() #Esta es la forma recomendada

+----------------+----+
|            name|  id|
+----------------+----+
|       Ana Perez|1001|
|      Luis Gomez|1002|
|     Maria Lopez|1003|
|  Javier Sanchez|1004|
|Carmen Rodriguez|1005|
+----------------+----+



La forma más concisa de seleccionar columnas es con los corchetes, pero dentro del estilo de programación de PySpark, a veces es más legible usar **select**. Queda a criterio del desarrollador cuál usar en cada momento.

In [100]:
df.select(F.col('name'),F.col('id')).show() #Esta es la forma recomendada

+----------------+----+
|            name|  id|
+----------------+----+
|       Ana Perez|1001|
|      Luis Gomez|1002|
|     Maria Lopez|1003|
|  Javier Sanchez|1004|
|Carmen Rodriguez|1005|
+----------------+----+



# Filter rows

**filter** también es una **transformación**.

In [101]:
df.filter(F.col('degree')=='Engineering').show()

+----+-----------+---+-----------+-----+
|  id|       name|age|     degree|grade|
+----+-----------+---+-----------+-----+
|1001|  Ana Perez| 20|Engineering|  8.5|
|1002| Luis Gomez| 22|Engineering|  9.0|
|1003|Maria Lopez| 21|Engineering|  7.8|
+----+-----------+---+-----------+-----+



In [102]:
df.filter(F.col('degree')=='Engineering').show()

+----+-----------+---+-----------+-----+
|  id|       name|age|     degree|grade|
+----+-----------+---+-----------+-----+
|1001|  Ana Perez| 20|Engineering|  8.5|
|1002| Luis Gomez| 22|Engineering|  9.0|
|1003|Maria Lopez| 21|Engineering|  7.8|
+----+-----------+---+-----------+-----+



Se pueden concatenar varias condiciones indicándolas entre **paréntesis**.

In [103]:
df.filter((F.col('degree')=='Engineering') & (F.col('grade')>8)).show()

+----+----------+---+-----------+-----+
|  id|      name|age|     degree|grade|
+----+----------+---+-----------+-----+
|1001| Ana Perez| 20|Engineering|  8.5|
|1002|Luis Gomez| 22|Engineering|  9.0|
+----+----------+---+-----------+-----+



**Recuerda**: para que una operación aplique sobre el DataFrame que estamos usando, debemos asignar el resultado después de la operación.
```
df = df.filter(F.col('degree') == 'Engineering')
```

Además, ten encuenta lo que devuelve cada operación que aplicas. **filter** devuelve un DataFrame filtrado, pero **show** no devuelve nada, así que si asignas el resultado de **show** a tu variable, la variable valdrá **None**.
```
df = df.filter(F.col('degree') == 'Engineering').show()  # Ahora df = None
```

# Create and drop columns

In [104]:
df= df.withColumn('stars', F.col('grade')/2)
df.show()

+----+----------------+---+------------+-----+------------------+
|  id|            name|age|      degree|grade|             stars|
+----+----------------+---+------------+-----+------------------+
|1001|       Ana Perez| 20| Engineering|  8.5|              4.25|
|1002|      Luis Gomez| 22| Engineering|  9.0|               4.5|
|1003|     Maria Lopez| 21| Engineering|  7.8|3.9000000953674316|
|1004|  Javier Sanchez| 23|Architecture|  6.5|              3.25|
|1005|Carmen Rodriguez| 20|Architecture|  8.2| 4.099999904632568|
+----+----------------+---+------------+-----+------------------+



Se pueden concatenar operaciones y hacer referencia a columnas creadas en operaciones previas.

In [105]:
df= df.withColumn('stars', F.col('grade')/2)\
.withColumn('stars',F.round(F.col('stars')))#De esta manera evitamos la concatenación de variables
df.show()

+----+----------------+---+------------+-----+-----+
|  id|            name|age|      degree|grade|stars|
+----+----------------+---+------------+-----+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.0|
|1002|      Luis Gomez| 22| Engineering|  9.0|  5.0|
|1003|     Maria Lopez| 21| Engineering|  7.8|  4.0|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.0|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.0|
+----+----------------+---+------------+-----+-----+



Se pueden anidar funciones unas dentro de otras

In [106]:
df.withColumn('first_name',F.split(F.col('name'),' ')[0]).show()

+----+----------------+---+------------+-----+-----+----------+
|  id|            name|age|      degree|grade|stars|first_name|
+----+----------------+---+------------+-----+-----+----------+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.0|       Ana|
|1002|      Luis Gomez| 22| Engineering|  9.0|  5.0|      Luis|
|1003|     Maria Lopez| 21| Engineering|  7.8|  4.0|     Maria|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.0|    Javier|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.0|    Carmen|
+----+----------------+---+------------+-----+-----+----------+



In [107]:
df.withColumn('first_name',F.split(F.col('name'),' ')[0])\
.withColumn('last_name',F.split(F.col('name'),' ')[1]).show()

+----+----------------+---+------------+-----+-----+----------+---------+
|  id|            name|age|      degree|grade|stars|first_name|last_name|
+----+----------------+---+------------+-----+-----+----------+---------+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.0|       Ana|    Perez|
|1002|      Luis Gomez| 22| Engineering|  9.0|  5.0|      Luis|    Gomez|
|1003|     Maria Lopez| 21| Engineering|  7.8|  4.0|     Maria|    Lopez|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.0|    Javier|  Sanchez|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.0|    Carmen|Rodriguez|
+----+----------------+---+------------+-----+-----+----------+---------+



In [108]:
df.withColumn('first_name',F.split(F.col('name'),' ')[0])\
.withColumn('last_name',F.split(F.col('name'),' ')[1])\
.drop('name').show()

+----+---+------------+-----+-----+----------+---------+
|  id|age|      degree|grade|stars|first_name|last_name|
+----+---+------------+-----+-----+----------+---------+
|1001| 20| Engineering|  8.5|  4.0|       Ana|    Perez|
|1002| 22| Engineering|  9.0|  5.0|      Luis|    Gomez|
|1003| 21| Engineering|  7.8|  4.0|     Maria|    Lopez|
|1004| 23|Architecture|  6.5|  3.0|    Javier|  Sanchez|
|1005| 20|Architecture|  8.2|  4.0|    Carmen|Rodriguez|
+----+---+------------+-----+-----+----------+---------+



# Group by

**groupBy** también es una transformación que se ejecuta de forma **lazy**

In [109]:
df.groupBy('degree').count().show()

+------------+-----+
|      degree|count|
+------------+-----+
| Engineering|    3|
|Architecture|    2|
+------------+-----+



In [110]:
df.groupBy('degree').agg(F.avg(F.col('stars'))).show() #Anteriormente para count() no era necesario agg porque count es una operación muy sencilla

+------------+-----------------+
|      degree|       avg(stars)|
+------------+-----------------+
| Engineering|4.333333333333333|
|Architecture|              3.5|
+------------+-----------------+



Se pueden aplicar varios cálculos de agregación

In [111]:
df.groupBy('degree').agg(F.avg(F.col('stars')))
new_df= df.withColumnRenamed('avg(stars)','avg_stars').show()

+----+----------------+---+------------+-----+-----+
|  id|            name|age|      degree|grade|stars|
+----+----------------+---+------------+-----+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.0|
|1002|      Luis Gomez| 22| Engineering|  9.0|  5.0|
|1003|     Maria Lopez| 21| Engineering|  7.8|  4.0|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.0|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.0|
+----+----------------+---+------------+-----+-----+



In [112]:
df.groupBy('degree').agg(F.avg(F.col('stars')).alias('avg_stars')).show()

+------------+-----------------+
|      degree|        avg_stars|
+------------+-----------------+
| Engineering|4.333333333333333|
|Architecture|              3.5|
+------------+-----------------+



In [113]:
df.groupBy('degree').agg(
  F.avg(F.col('stars')).alias('avg_stars'),
  F.min(F.col('stars')).alias('stars_min'),
  F.max(F.col('stars')).alias('stars_max')
).show()

+------------+-----------------+---------+---------+
|      degree|        avg_stars|stars_min|stars_max|
+------------+-----------------+---------+---------+
| Engineering|4.333333333333333|      4.0|      5.0|
|Architecture|              3.5|      3.0|      4.0|
+------------+-----------------+---------+---------+



# Join

In [114]:
data = [

    (1001, "Huelva"),

    (1002, "Sevilla"),

    (1003, "Huelva"),

    (1004, "Sevilla"),

    (1005, "Huelva")

]

schema = T.StructType([

    T.StructField("id", T.IntegerType(), True),

    T.StructField("city", T.StringType(), True)

])

df_city = spark.createDataFrame(data, schema)

df_city.show()



+----+-------+
|  id|   city|
+----+-------+
|1001| Huelva|
|1002|Sevilla|
|1003| Huelva|
|1004|Sevilla|
|1005| Huelva|
+----+-------+



In [115]:
df.show()

+----+----------------+---+------------+-----+-----+
|  id|            name|age|      degree|grade|stars|
+----+----------------+---+------------+-----+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.0|
|1002|      Luis Gomez| 22| Engineering|  9.0|  5.0|
|1003|     Maria Lopez| 21| Engineering|  7.8|  4.0|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.0|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.0|
+----+----------------+---+------------+-----+-----+



In [116]:
df.join(df_city, on='id',how='inner').show() #inner significa que si tenemos un estudiante en el df_city pero no esta presente en df, ese mismo no se muestra

+----+----------------+---+------------+-----+-----+-------+
|  id|            name|age|      degree|grade|stars|   city|
+----+----------------+---+------------+-----+-----+-------+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.0| Huelva|
|1002|      Luis Gomez| 22| Engineering|  9.0|  5.0|Sevilla|
|1003|     Maria Lopez| 21| Engineering|  7.8|  4.0| Huelva|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.0|Sevilla|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.0| Huelva|
+----+----------------+---+------------+-----+-----+-------+



In [121]:
df=df.join(df_city, on='id',how='inner')
df.show()

+----+----------------+---+------------+-----+-----+-------+
|  id|            name|age|      degree|grade|stars|   city|
+----+----------------+---+------------+-----+-----+-------+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.0| Huelva|
|1002|      Luis Gomez| 22| Engineering|  9.0|  5.0|Sevilla|
|1003|     Maria Lopez| 21| Engineering|  7.8|  4.0| Huelva|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.0|Sevilla|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.0| Huelva|
+----+----------------+---+------------+-----+-----+-------+



# Write

El resultado de esta escritura será una carpeta llamada **students.csv** que contendrá partes de un csv

In [117]:
df.write.csv('/content/data3/csv/student.csv')

En Spark se indican las rutas para leer o escribir se indican a nivel de **carpeta**, no de fichero

In [118]:
df.write.csv('/content/data3/csv/student')

Automáticamente, Spark escribe la cantidad de ficheros que considere óptima. El equilibrio entre cantidad de **ficheros** y **tamaño de fichero**, es algo crucial en problemas de Big Data para conseguir un **rendimiento**.

In [119]:
df_new=spark.read.csv('/content/data3/csv/student')
df_new.show()


+----+----------------+---+------------+---+---+
| _c0|             _c1|_c2|         _c3|_c4|_c5|
+----+----------------+---+------------+---+---+
|1003|     Maria Lopez| 21| Engineering|7.8|4.0|
|1004|  Javier Sanchez| 23|Architecture|6.5|3.0|
|1005|Carmen Rodriguez| 20|Architecture|8.2|4.0|
|1001|       Ana Perez| 20| Engineering|8.5|4.0|
|1002|      Luis Gomez| 22| Engineering|9.0|5.0|
+----+----------------+---+------------+---+---+



El particionado es otro aspecto crítico en problemas de Big Data. Permite filtrar de forma más eficiente un DataFrame, permitiendo cargar únicamente la partición de interés. Si bien, también se debe **balancear el número de particiones y la profundidad de las particiones** para tener un buen rendimiento.

Para ello, se debe saber de antemano **cómo se va a consumir la información** y en base a ello, seleccionar las columnas con las que particionar.

In [122]:
df.write.partitionBy('city').csv('/content/data3/csv/student_partitioned')


In [123]:
df.write.partitionBy('city','degree').csv('/content/data3/csv/student_partitioned_v2')

El formato de fichero habitual para trabajar con Spark es **parquet**. Es un formato **binario** optimizado para Spark que permite comprimir la información hasta más de un 90% y leer partes del fichero sin tener que cargarlo al completo. Parquet es un fichero binario.

In [124]:
df.write.parquet('content/data3/parquet/students')

Se puede particionar por más de una columna indicándola en el **partitionBy**

# Read

Observa que al haber leído a nivel de partición, la tabla no tiene la columna **city**. Esto se debe a que la **columna de partición no se guarda en el fichero** porque se puede deducir de la partición. Así, se ahorra espacio en disco.

Al indicar la ruta raíz (**students**), Spark lee todos las particiones y ficheros y los concatena automáticamente en una sola tabla. De esta forma el desarrollador se despreocupa de concatenar manualmente cada fichero y cada partición.